# Text generation using a RNN

Given a sequence of words from this data, train a model to predict the next word in the sequence. Longer sequences of text can be generated by calling the model repeatedly.

**Mount your Google Drive**

### Import Keras and other libraries

In [1]:
import glob

from sklearn.utils import shuffle
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam
from keras import backend

F:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Download data
Reference: Data is collected from http://www.gutenberg.org

For the lab purpose, you can load the dataset provided by Great Learning

### Load the Oscar Wilde dataset

Store all the ".txt" file names in a list

In [60]:
All_text = glob.glob("data\\*.txt")

In [61]:
All_text

['data\\A Critic in Pall Mall.txt',
 'data\\A House of Pomegranates.txt',
 'data\\A Woman of No Importance a play.txt',
 'data\\An Ideal Husband.txt',
 'data\\Charmides and Other Poems.txt',
 'data\\Children in Prison and Other Cruelties of Prison Life.txt',
 'data\\De Profundis.txt',
 'data\\Essays and Lectures.txt',
 'data\\For Love of the King.txt',
 'data\\Impressions of America.txt',
 'data\\Intentions.txt',
 'data\\Lady Windermere_s Fan.txt',
 'data\\Lord Arthur Savile_s Crime.txt',
 'data\\Miscellaneous Aphorisms_ The Soul of Man.txt',
 'data\\Miscellanies.txt',
 'data\\Oscar Wilde Miscellaneous.txt',
 'data\\Poems with the Ballad of Reading Gaol.txt',
 'data\\Reviews.txt',
 'data\\Rose Leaf and Apple Leaf.txt',
 'data\\Salomé A tragedy in one act.txt',
 'data\\Selected poems of oscar wilde including The Ballad of Reading Gaol.txt',
 'data\\Selected prose of oscar wilde with a Preface by Robert Ross.txt',
 'data\\Shorter Prose Pieces.txt',
 'data\\The Ballad of Reading Gaol.txt'

### Read the data

Read contents of every file from the list and append the text in a new list

In [62]:
codetext = []
bookranges = []
for all_txt in All_text:
    txt = open(all_txt, "r",encoding="utf8")
    start = len(codetext)
    codetext.append(txt.read())
    end = len(codetext)
    bookranges.append({"start": start, "end": end, "name": all_txt.rsplit("/", 1)[-1]})
    txt.close()
    #print (readd)

## Process the text
Initialize and fit the tokenizer

In [63]:
tokenizer = Tokenizer(lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(codetext)

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping words to numbers, and another for numbers to words.

In [64]:
word_idx = tokenizer.word_index
idx_word = tokenizer.index_word

Get the word count for every word and also get the total number of words.

In [65]:
word_counts = tokenizer.word_counts
num_words = len(word_idx) + 1

Convert text to sequence of numbers

In [66]:
sequences = tokenizer.texts_to_sequences(codetext)

### Generate Features and Labels

In [67]:
features = []
labels = []

training_length = 50
# Iterate through the sequences of tokens
for seq in sequences:
    # Create multiple training examples from each sequence
    for i in range(training_length, training_length+300):
        # Extract the features and label
        extract = seq[i - training_length: i - training_length + 20]

        # Set the features and label
        features.append(extract[:-1])
        labels.append(extract[-1])

### The prediction task

Given a word, or a sequence of words, what is the most probable next word? This is the task we're training the model to perform. The input to the model will be a sequence of words, and we train the model to predict the output—the following word at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the words computed until this moment, what is the next word?

### Generate training and testing data

In [68]:
from sklearn.utils import shuffle
import numpy as np

features, labels = shuffle(features, labels, random_state=1)

# Decide on number of samples for training
train_end = int(0.7 * len(labels))

train_features = np.array(features[:train_end])
valid_features = np.array(features[train_end:])

train_labels = labels[:train_end]
valid_labels = labels[train_end:]

# Convert to arrays
X_train, X_valid = np.array(train_features), np.array(valid_features)

# Using int8 for memory savings
y_train = np.zeros((len(train_labels), num_words), dtype=np.int8)
y_valid = np.zeros((len(valid_labels), num_words), dtype=np.int8)

# One hot encoding of labels
for example_index, word_index in enumerate(train_labels):
    y_train[example_index, word_index] = 1

for example_index, word_index in enumerate(valid_labels):
    y_valid[example_index, word_index] = 1

This is just to check the features and labels

In [69]:
for i, sequence in enumerate(X_train[:2]):
    text = []
#     print(i, sequence)
    for idx in sequence:
        text.append(idx_word[idx])
        
    print('Features: ' + ' '.join(text)+'\n')
    print('Label: ' + idx_word[np.argmax(y_train[i])] + '\n')

Features: 25 minutes act ii room in the duke’s palace 36 minutes act iii corridor in the duke’s palace 29

Label: minutes

Features: email ccx074 pglaf org note that later editions of de profundis contained more material the most complete editions are

Label: still



## Build The Model

Use `keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `keras.layers.LSTM`: A type of RNN with size `units=rnn_units` (You can also use a GRU layer here.)
* `keras.layers.Dense`: The output layer, with `num_words` outputs.

In [70]:
model = Sequential()

# Embedding layer
model.add(
    Embedding(
        input_dim=len(word_idx) + 1,
        output_dim=100,
        weights=None,
        trainable=True))

# Recurrent layer
model.add(
    LSTM(
        64, return_sequences=False, dropout=0.1,
        recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         3283900   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32839)             2134535   
Total params: 5,464,835
Trainable params: 5,464,835
Non-trainable params: 0
_________________________________________________________________


For each word the model looks up the embedding, runs the LSTM one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next word.

## Train the model

In [72]:
h = model.fit(X_train, y_train, epochs = 200, batch_size = 50, 
          verbose = 1)## Train the model

Epoch 1/300
6510/6510 [==============================] - 27s 4ms/step - loss: 6.6859 - acc: 0.0598
Epoch 2/300
6510/6510 [==============================] - 27s 4ms/step - loss: 6.4426 - acc: 0.0582
Epoch 3/300
6510/6510 [==============================] - 27s 4ms/step - loss: 6.2977 - acc: 0.0575
Epoch 4/300
6510/6510 [==============================] - 28s 4ms/step - loss: 6.1728 - acc: 0.0590
Epoch 5/300
6510/6510 [==============================] - 27s 4ms/step - loss: 6.0622 - acc: 0.0628
Epoch 6/300
6510/6510 [==============================] - 27s 4ms/step - loss: 5.9683 - acc: 0.0639
Epoch 7/300
6510/6510 [==============================] - 27s 4ms/step - loss: 5.8579 - acc: 0.0676
Epoch 8/300
6510/6510 [==============================] - 27s 4ms/step - loss: 5.7487 - acc: 0.0713
Epoch 9/300
6510/6510 [==============================] - 27s 4ms/step - loss: 5.6688 - acc: 0.0747
Epoch 10/300
6510/6510 [==============================] - 27s 4ms/step - loss: 5.5786 - acc: 0.0754
Epoch 11/

6510/6510 [==============================] - 26s 4ms/step - loss: 1.1821 - acc: 0.6762
Epoch 84/300
6510/6510 [==============================] - 26s 4ms/step - loss: 1.1640 - acc: 0.6742
Epoch 85/300
6510/6510 [==============================] - 26s 4ms/step - loss: 1.1333 - acc: 0.6837
Epoch 86/300
6510/6510 [==============================] - 26s 4ms/step - loss: 1.1598 - acc: 0.6736
Epoch 87/300
6510/6510 [==============================] - 26s 4ms/step - loss: 1.1604 - acc: 0.6773
Epoch 88/300
6510/6510 [==============================] - 26s 4ms/step - loss: 1.1319 - acc: 0.6849
Epoch 89/300
6510/6510 [==============================] - 26s 4ms/step - loss: 1.1102 - acc: 0.6896
Epoch 90/300
6510/6510 [==============================] - 26s 4ms/step - loss: 1.1041 - acc: 0.6952
Epoch 91/300
6510/6510 [==============================] - 26s 4ms/step - loss: 1.0889 - acc: 0.6934
Epoch 92/300
6510/6510 [==============================] - 26s 4ms/step - loss: 1.0745 - acc: 0.7014
Epoch 93/300


6510/6510 [==============================] - 29s 5ms/step - loss: 0.6458 - acc: 0.8034
Epoch 165/300
6510/6510 [==============================] - 33s 5ms/step - loss: 0.6513 - acc: 0.8083
Epoch 166/300
6510/6510 [==============================] - 33s 5ms/step - loss: 0.6473 - acc: 0.8009
Epoch 167/300
6510/6510 [==============================] - 34s 5ms/step - loss: 0.6427 - acc: 0.8041
Epoch 168/300
6510/6510 [==============================] - 33s 5ms/step - loss: 0.6280 - acc: 0.8152
Epoch 169/300
6510/6510 [==============================] - 34s 5ms/step - loss: 0.6167 - acc: 0.8127
Epoch 170/300
6510/6510 [==============================] - 35s 5ms/step - loss: 0.6196 - acc: 0.8126
Epoch 171/300
6510/6510 [==============================] - 30s 5ms/step - loss: 0.6275 - acc: 0.8065
Epoch 172/300
6510/6510 [==============================] - 29s 5ms/step - loss: 0.6405 - acc: 0.8048
Epoch 173/300
6510/6510 [==============================] - 30s 5ms/step - loss: 0.6115 - acc: 0.8104
Epoc

KeyboardInterrupt: 

### Save Model

In [54]:
# save the model to file
model.save('model_R9_Ext.h5')

## If you have already trained the model and saved it, you can load a pretrained model

In [56]:
model = load_model('model_R9_Ext.h5')

### Note: After loading the model run  model.fit()  to continue training form there, if required.

In [57]:
model.fit(X_train, y_train, batch_size=50, epochs=300)

Epoch 1/700
210/210 [==============================] - ETA: 0s - loss: 0.5745 - acc: 0.840 - 1s 6ms/step - loss: 0.5705 - acc: 0.8381
Epoch 2/700
210/210 [==============================] - 0s 2ms/step - loss: 0.6124 - acc: 0.8143
Epoch 3/700
210/210 [==============================] - 0s 2ms/step - loss: 0.6230 - acc: 0.8095
Epoch 4/700
210/210 [==============================] - 0s 2ms/step - loss: 0.6111 - acc: 0.8476
Epoch 5/700
210/210 [==============================] - 0s 2ms/step - loss: 0.6192 - acc: 0.8476
Epoch 6/700
210/210 [==============================] - 0s 2ms/step - loss: 0.6351 - acc: 0.8095
Epoch 7/700
210/210 [==============================] - 0s 2ms/step - loss: 0.6377 - acc: 0.8190
Epoch 8/700
210/210 [==============================] - 0s 2ms/step - loss: 0.6725 - acc: 0.7905
Epoch 9/700
210/210 [==============================] - 0s 2ms/step - loss: 0.6169 - acc: 0.8238
Epoch 10/700
210/210 [==============================] - 0s 2ms/step - loss: 0.6410 - acc: 0.8238
E

210/210 [==============================] - 0s 2ms/step - loss: 0.4028 - acc: 0.8857
Epoch 86/700
210/210 [==============================] - 0s 2ms/step - loss: 0.4326 - acc: 0.8714
Epoch 87/700
210/210 [==============================] - 0s 2ms/step - loss: 0.4539 - acc: 0.8524
Epoch 88/700
210/210 [==============================] - 0s 2ms/step - loss: 0.4218 - acc: 0.8619
Epoch 89/700
210/210 [==============================] - 0s 2ms/step - loss: 0.4633 - acc: 0.8571
Epoch 90/700
210/210 [==============================] - 0s 2ms/step - loss: 0.4476 - acc: 0.8952
Epoch 91/700
210/210 [==============================] - 0s 2ms/step - loss: 0.4488 - acc: 0.8619
Epoch 92/700
210/210 [==============================] - 0s 2ms/step - loss: 0.5089 - acc: 0.8190
Epoch 93/700
210/210 [==============================] - 0s 2ms/step - loss: 0.4698 - acc: 0.8571
Epoch 94/700
210/210 [==============================] - 0s 2ms/step - loss: 0.4331 - acc: 0.8714
Epoch 95/700
210/210 [=====================

210/210 [==============================] - 0s 2ms/step - loss: 0.3790 - acc: 0.8714
Epoch 169/700
210/210 [==============================] - 0s 2ms/step - loss: 0.3773 - acc: 0.9048
Epoch 170/700
210/210 [==============================] - 0s 2ms/step - loss: 0.3884 - acc: 0.8619
Epoch 171/700
210/210 [==============================] - 0s 2ms/step - loss: 0.3699 - acc: 0.8810
Epoch 172/700
210/210 [==============================] - 0s 2ms/step - loss: 0.4255 - acc: 0.8714
Epoch 173/700
210/210 [==============================] - 0s 2ms/step - loss: 0.3650 - acc: 0.8762
Epoch 174/700
210/210 [==============================] - 0s 2ms/step - loss: 0.3651 - acc: 0.8762
Epoch 175/700
210/210 [==============================] - 0s 2ms/step - loss: 0.3586 - acc: 0.8762
Epoch 176/700
210/210 [==============================] - 0s 2ms/step - loss: 0.4143 - acc: 0.8619
Epoch 177/700
210/210 [==============================] - 0s 2ms/step - loss: 0.3245 - acc: 0.9000
Epoch 178/700
210/210 [===========

210/210 [==============================] - 0s 2ms/step - loss: 0.3056 - acc: 0.9238
Epoch 252/700
210/210 [==============================] - 0s 2ms/step - loss: 0.3153 - acc: 0.9143
Epoch 253/700
210/210 [==============================] - 0s 2ms/step - loss: 0.3138 - acc: 0.9095
Epoch 254/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2985 - acc: 0.9190
Epoch 255/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2863 - acc: 0.9238
Epoch 256/700
210/210 [==============================] - 0s 2ms/step - loss: 0.3198 - acc: 0.8952
Epoch 257/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2786 - acc: 0.9143
Epoch 258/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2684 - acc: 0.9381
Epoch 259/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2908 - acc: 0.9048
Epoch 260/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2988 - acc: 0.9095
Epoch 261/700
210/210 [===========

210/210 [==============================] - 0s 2ms/step - loss: 0.2835 - acc: 0.8905
Epoch 335/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2576 - acc: 0.9143
Epoch 336/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2920 - acc: 0.8810
Epoch 337/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2363 - acc: 0.9571
Epoch 338/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2717 - acc: 0.9048
Epoch 339/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2271 - acc: 0.9381
Epoch 340/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2981 - acc: 0.9095
Epoch 341/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2704 - acc: 0.8952
Epoch 342/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2763 - acc: 0.9000
Epoch 343/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1995 - acc: 0.9571
Epoch 344/700
210/210 [===========

210/210 [==============================] - 0s 2ms/step - loss: 0.2404 - acc: 0.9381
Epoch 418/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1768 - acc: 0.9619
Epoch 419/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2198 - acc: 0.9333
Epoch 420/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1779 - acc: 0.9571
Epoch 421/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2294 - acc: 0.9238
Epoch 422/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2047 - acc: 0.9524
Epoch 423/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1861 - acc: 0.9429
Epoch 424/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2237 - acc: 0.9095
Epoch 425/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2029 - acc: 0.9190
Epoch 426/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2621 - acc: 0.9238
Epoch 427/700
210/210 [===========

210/210 [==============================] - 0s 2ms/step - loss: 0.1847 - acc: 0.9381
Epoch 501/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1765 - acc: 0.9524
Epoch 502/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1729 - acc: 0.9381
Epoch 503/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1892 - acc: 0.9381
Epoch 504/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1764 - acc: 0.9476
Epoch 505/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1671 - acc: 0.9476
Epoch 506/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1946 - acc: 0.9333
Epoch 507/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2058 - acc: 0.9333
Epoch 508/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1800 - acc: 0.9476
Epoch 509/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1937 - acc: 0.9381
Epoch 510/700
210/210 [===========

210/210 [==============================] - 0s 2ms/step - loss: 0.2148 - acc: 0.9381
Epoch 584/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1673 - acc: 0.9476
Epoch 585/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1342 - acc: 0.9714
Epoch 586/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1717 - acc: 0.9476
Epoch 587/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1934 - acc: 0.9286
Epoch 588/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1455 - acc: 0.9524
Epoch 589/700
210/210 [==============================] - 0s 2ms/step - loss: 0.2029 - acc: 0.9429
Epoch 590/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1850 - acc: 0.9333
Epoch 591/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1815 - acc: 0.9524
Epoch 592/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1478 - acc: 0.9667
Epoch 593/700
210/210 [===========

210/210 [==============================] - 0s 2ms/step - loss: 0.1281 - acc: 0.9667
Epoch 667/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1675 - acc: 0.9333
Epoch 668/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1233 - acc: 0.9667
Epoch 669/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1250 - acc: 0.9619
Epoch 670/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1521 - acc: 0.9524
Epoch 671/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1522 - acc: 0.9429
Epoch 672/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1974 - acc: 0.9286
Epoch 673/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1918 - acc: 0.9333
Epoch 674/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1899 - acc: 0.9524
Epoch 675/700
210/210 [==============================] - 0s 2ms/step - loss: 0.1835 - acc: 0.9381
Epoch 676/700
210/210 [===========

## Evaluation

In [58]:
print(model.evaluate(X_train, y_train, batch_size = 20))
print('\nModel Performance: Log Loss and Accuracy on validation data')
print(model.evaluate(X_valid, y_valid, batch_size = 20))

210/210 [==============================] - 0s 2ms/step
[0.018749392846421825, 0.9952380941027686]

Model Performance: Log Loss and Accuracy on validation data
90/90 [==============================] - 0s 421us/step
[14.03720039791531, 0.06666666766007741]


## Generate text

In [59]:
seed_length=50
new_words=50
diversity=1
n_gen=1

import random

# Choose a random sequence
seq = random.choice(sequences)

# print seq

# Choose a random starting point
seed_idx = random.randint(0, len(seq) - seed_length - 10)
# Ending index for seed
end_idx = seed_idx + seed_length

gen_list = []

for n in range(n_gen):
    # Extract the seed sequence
    seed = seq[seed_idx:end_idx]
    original_sequence = [idx_word[i] for i in seed]
    generated = seed[:] + ['#']

    # Find the actual entire sequence
    actual = generated[:] + seq[end_idx:end_idx + new_words]
        
    # Keep adding new words
    for i in range(new_words):

        # Make a prediction from the seed
        preds = model.predict(np.array(seed).reshape(1, -1))[0].astype(np.float64)

        # Diversify
        preds = np.log(preds) / diversity
        exp_preds = np.exp(preds)

        # Softmax
        preds = exp_preds / sum(exp_preds)

        # Choose the next word
        probas = np.random.multinomial(1, preds, 1)[0]

        next_idx = np.argmax(probas)

        # New seed adds on old word
        #             seed = seed[1:] + [next_idx]
        seed += [next_idx]
        generated.append(next_idx)
    # Showing generated and actual abstract
    n = []

    for i in generated:
        n.append(idx_word.get(i, '< --- >'))

    gen_list.append(n)

a = []

for i in actual:
    a.append(idx_word.get(i, '< --- >'))

a = a[seed_length:]

gen_list = [gen[seed_length:seed_length + len(a)] for gen in gen_list]

print('Original Sequence: \n'+' '.join(original_sequence))
print("\n")
# print(gen_list)
print('Generated Sequence: \n'+' '.join(gen_list[0][1:]))
# print(a)

F:\Anaconda\lib\site-packages\ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in log


Original Sequence: 
a sudden vast terrible direct and indirect stimulus for new and national declamatory expression had not been given to me it is certain i say that although i had made a start before only from the occurrence of the secession war and what it show’d me as by flashes of


Generated Sequence: 
the tomb of keats tomb of anyone a critic in england the poetry of keats letters of the people 29 ltd edition by oscar wilde editor e v lucas release date october 6 2009 ebook is the afternoon tea on george at www christian art christian art madame ristori 85
